In [1]:
import tensorflow as tf

In [2]:
print(tf.__version__)

1.13.1


In [3]:
x = tf.Variable(3, name = "x")
y = tf.Variable(4, name = "y")
f = x*x*y + y + 2

Instructions for updating:
Colocations handled automatically by placer.


In [4]:
session = tf.Session()
session.run(x.initializer)
session.run(y.initializer)
result = session.run(f)
print(result)
session.close()

42


In [5]:
with tf.Session() as session:
    x.initializer.run()
    y.initializer.run()
    result = f.eval()
    print(result)

42


In [6]:
init = tf.global_variables_initializer()

with tf.Session() as session:
    init.run()
    result = f.eval()
    print(result)

42


In [7]:
session = tf.InteractiveSession()
init.run()
result = f.eval()
print(result)
session.close()

42


In [8]:
x1 = tf.Variable(1)

In [9]:
x1.graph

In [10]:
x2 = tf.Variable(2)

In [11]:
x2.graph

In [12]:
graph = tf.Graph()
with graph.as_default():
    x2 = tf.Variable(2)

In [13]:
x2.graph

In [14]:
x2.graph is tf.get_default_graph()

False

In [15]:
tf.reset_default_graph()

In [16]:
w = tf.constant(3)
x = w + 2
y = x + 5
z = x * 3

In [17]:
with tf.Session() as session:
    print(y.eval()) #10
    print(z.eval()) #15

10
15


In [18]:
# evaluate w and x once
with tf.Session() as session:
    y_val, z_val = session.run([y, z])
    print(y_val) # 10
    print(z_val) # 15

10
15


In [19]:
import numpy as np
from sklearn.datasets import fetch_california_housing

housing = fetch_california_housing()
m, n = housing.data.shape
housing_data_plus_bias = np.c_[np.ones((m, 1)), housing.data]

X = tf.constant(housing_data_plus_bias, dtype = tf.float32, name = "X")
y = tf.constant(housing.target.reshape(-1, 1), dtype = tf.float32, name = "y")
XT = tf.transpose(X)
theta = tf.matmul(tf.matmul(tf.matrix_inverse(tf.matmul(XT, X)), XT), y) # theta = (XT * X)^(-1) * XT * y

with tf.Session() as session:
    theta_value = theta.eval()

In [20]:
theta_value

array([[-3.6845909e+01],
       [ 4.3667579e-01],
       [ 9.4581824e-03],
       [-1.0708507e-01],
       [ 6.4334780e-01],
       [-3.9223705e-06],
       [-3.7883162e-03],
       [-4.2040566e-01],
       [-4.3347809e-01]], dtype=float32)

In [21]:
from sklearn.preprocessing import StandardScaler

n_epochs = 1000
learning_rate = 0.01

standard_scaler = StandardScaler()
scaled_housing_data_plus_bias = standard_scaler.fit_transform(housing_data_plus_bias)

In [22]:
%%time
X = tf.constant(scaled_housing_data_plus_bias, dtype = tf.float32, name = "X")
y = tf.constant(housing.target.reshape(-1, 1), dtype = tf.float32, name = "y")
theta = tf.Variable(tf.random_uniform([n + 1, 1], -1.0, 1.0), name = "theta")
y_pred = tf.matmul(X, theta, name = "predictions")
error = y_pred - y
mse = tf.reduce_mean(tf.square(error), name = "mse")
gradients = 2 / m * tf.matmul(tf.transpose(X), error)
training_op = tf.assign(theta, theta - learning_rate * gradients)

init = tf.global_variables_initializer()

with tf.Session() as session:
    session.run(init)
    
    for epoch in range(n_epochs):
        if epoch % 100 == 0:
            print("Epoch", epoch, "MSE = ", mse.eval())
        session.run(training_op)
        
    best_theta = theta.eval()

Epoch 0 MSE =  10.5321045
Epoch 100 MSE =  5.1339684
Epoch 200 MSE =  4.995699
Epoch 300 MSE =  4.94221
Epoch 400 MSE =  4.9052615
Epoch 500 MSE =  4.8784156
Epoch 600 MSE =  4.858794
Epoch 700 MSE =  4.84442
Epoch 800 MSE =  4.833868
Epoch 900 MSE =  4.826103
Wall time: 695 ms


In [23]:
%%time
X = tf.constant(scaled_housing_data_plus_bias, dtype = tf.float32, name = "X")
y = tf.constant(housing.target.reshape(-1, 1), dtype = tf.float32, name = "y")
theta = tf.Variable(tf.random_uniform([n + 1, 1], -1.0, 1.0), name = "theta")
y_pred = tf.matmul(X, theta, name = "predictions")
error = y_pred - y
mse = tf.reduce_mean(tf.square(error), name = "mse")
gradients = tf.gradients(mse, [theta])[0]
training_op = tf.assign(theta, theta - learning_rate * gradients)

init = tf.global_variables_initializer()

with tf.Session() as session:
    session.run(init)
    
    for epoch in range(n_epochs):
        if epoch % 100 == 0:
            print("Epoch", epoch, "MSE = ", mse.eval())
        session.run(training_op)
        
    best_theta = theta.eval()

Epoch 0 MSE =  7.8109665
Epoch 100 MSE =  4.8669333
Epoch 200 MSE =  4.8320637
Epoch 300 MSE =  4.823446
Epoch 400 MSE =  4.8179
Epoch 500 MSE =  4.8139215
Epoch 600 MSE =  4.811036
Epoch 700 MSE =  4.8089404
Epoch 800 MSE =  4.8074174
Epoch 900 MSE =  4.806308
Wall time: 402 ms


In [24]:
%%time
X = tf.constant(scaled_housing_data_plus_bias, dtype = tf.float32, name = "X")
y = tf.constant(housing.target.reshape(-1, 1), dtype = tf.float32, name = "y")
theta = tf.Variable(tf.random_uniform([n + 1, 1], -1.0, 1.0), name = "theta")
y_pred = tf.matmul(X, theta, name = "predictions")
error = y_pred - y
mse = tf.reduce_mean(tf.square(error), name = "mse")
optimizer = tf.train.MomentumOptimizer(learning_rate=learning_rate, momentum = 0.9)

init = tf.global_variables_initializer()

Wall time: 8.98 ms


In [25]:
from tensorflow.python.client import device_lib

In [26]:
def get_available_gpu_list():
    local_device_protos = device_lib.list_local_devices()
    return [x.name for x in local_device_protos if x.device_type == 'GPU']

In [27]:
get_available_gpu_list()

['/device:GPU:0', '/device:GPU:1']